# FINAL PROJECT  - TWITTER CRAWLER


In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter
from config import *

In [2]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor
import json
import datetime

## 1. COLLECT DATA

In [3]:
## access token informations 
access_token1 = "989425972633063424-CikJNGMKQt7ABgiMdHD1DczF6Co57FF"
access_token_secret1 = "xcIEveiK9PYXlomq3hNOTfIcpbM8zWbLkwyp1ubYsEamp"

consumer_key1 = "5QDGnriy2sm3DGyd1vRjTG6um"
consumer_secret1 = "qXC4modanhi6LSNeoRxQCVm4I5TuisNrnTC7F0X84tHJcdbzJ2"

In [4]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler = auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [5]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [10]:
%%time

OUTPUT_FILENAME = "data/tweets.json"
stop_condition = 30000

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth = api.auth, listener = l)


# keywords we may want decide to track 
# covid, coronavirus, pandemic, virus, #covid, #covid19, #coronavirus
TRACKING_KEYWORDS = ["coronavirus", "covid", "pandemic", "virus",  "#covid", "#coronavirus", "#covid19"]
stream.filter(
    track = TRACKING_KEYWORDS, 
    is_async = False, 
    languages = ["en"]
)

Wall time: 26min 5s


## 2. LOAD DATA 

In [5]:
%%time

with open("data/tweets.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

Wall time: 22min 20s


In [6]:
%%time
df_tweets = pd.DataFrame.from_records(data)

Wall time: 10min 54s


In [7]:
print(df_tweets.shape)

(500526, 37)


## 3. STORE DATA TO CSV

In [8]:
df_tweets.to_csv("data/tweets.csv", index = False)

In [10]:
%%time
data = pd.read_csv("data/tweets.csv")

<c:\program files\python38\lib\site-packages\decorator.py:decorator-gen-62>:2: DtypeWarning: Columns (13,14,37) have mixed types.Specify dtype option on import or set low_memory=False.


Wall time: 5min 34s


In [11]:
data.head()

,Unnamed: 0,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,favorited,retweeted,filter_level,lang,timestamp_ms,display_text_range,extended_tweet,extended_entities,possibly_sensitive,withheld_in_countries
0,0,Sun Nov 22 10:54:27 +0000 2020,1330464655572164615,1330464655572164615,RT @SDPNorthEast: Travelling between Scotland ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,False,False,low,en,1606042467383,NaN,NaN,NaN,NaN,NaN
1,1,Sun Nov 22 10:54:27 +0000 2020,1330464656490721280,1330464656490721280,RT @SenKamalaHarris: I’m thankful for the farm...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,False,False,low,en,1606042467602,NaN,NaN,NaN,NaN,NaN
2,2,Sun Nov 22 10:54:27 +0000 2020,1330464656985628673,1330464656985628673,RT @caljspencer: A vaccine has been found for ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,False,False,low,en,1606042467720,NaN,NaN,NaN,NaN,NaN
3,3,Sun Nov 22 10:54:27 +0000 2020,1330464657086275585,1330464657086275585,Respect @NIkosdN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,...,False,False,low,en,1606042467744,NaN,NaN,NaN,NaN,NaN
4,4,Sun Nov 22 10:54:27 +0000 2020,1330464657069531137,1330464657069531137,@TheJasonPugh Putting out the message that 🇨🇦 ...,"<a href=""http://twitter.com/download/android"" ...",True,1.330229e+18,1.330229e+18,457136882.0,...,False,False,low,en,1606042467740,"[14, 140]","{'full_text': ""@TheJasonPugh Putting out the m...",NaN,NaN,NaN


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500526 entries, 0 to 500525
Data columns (total 38 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 500526 non-null  int64  
 1   created_at                 500526 non-null  object 
 2   id                         500526 non-null  int64  
 3   id_str                     500526 non-null  int64  
 4   text                       500526 non-null  object 
 5   source                     499071 non-null  object 
 6   truncated                  500526 non-null  bool   
 7   in_reply_to_status_id      54034 non-null   float64
 8   in_reply_to_status_id_str  54034 non-null   float64
 9   in_reply_to_user_id        58227 non-null   float64
 10  in_reply_to_user_id_str    58227 non-null   float64
 11  in_reply_to_screen_name    58223 non-null   object 
 12  user                       500526 non-null  object 
 13  geo                        73